In [2]:
import pandas as pd
import random
from typing import List, Dict, Optional, Set
import itertools

In [3]:
divisions = {
    "cin": ["plumpies", "fubar", "fearTheMex", "Teds"],
    "cod": ["pyong", "aguilar", "rum", "kaptain"],
    "cam": ["last", "aya", "tebro", "broseph"],
}
divisions

{'cin': ['plumpies', 'fubar', 'fearTheMex', 'Teds'],
 'cod': ['pyong', 'aguilar', 'rum', 'kaptain'],
 'cam': ['last', 'aya', 'tebro', 'broseph']}

In [941]:
def _get_other_div(team_name):
    team_to_div = _get_team_to_div()
    return list(itertools.chain.from_iterable([tms for div, tms in divisions.items() if div != team_to_div[team_name]]))    

def _get_opponent(team, must_play, team_subset=None) -> Dict[str, Set[str]]:
    """Select a match that has not been played by the team."""
    opponents = must_play[team]
    if team_subset:
        opponents = [o for o in opponents if o in team_subset]
    if opponents:                    
        other_team = list(opponents)[random.randint(0, len(opponents) - 1)]                 
        must_play[team].remove(other_team)
        must_play[other_team].remove(team)
        return other_team, must_play
    return None, must_play

def _get_team_to_div():
    team_to_div = {}
    for div_name, teams in divisions.items():
        for t in teams:
            team_to_div[t] = div_name
    return team_to_div

def _add_matchup(df, week_num, div_name, team, other_team):
    idx_num = len(df)
    df.loc[idx_num, "week"] = week_num
    df.loc[idx_num, "div"] = div_name
    df.loc[idx_num, "team_a"] = team
    df.loc[idx_num, "team_b"] = other_team

def _get_division_matches(
    teams: Dict[str, List[str]],
    div_name: str, 
    week_start: int,
):
    """Select divisional matchups."""
    played = dict((t, set()) for t in teams)
    df = pd.DataFrame()
    must_play = dict((t, set(teams) - {t}) for t in teams)
    
    for week_num in range(week_start, week_start + len(teams)):
        _teams_this_week = set(teams)
        while True:
            if not _teams_this_week:  # done with week
                break
            team = _teams_this_week.pop()
            while True:
                other_team, must_play = _get_opponent(team, must_play)
                if not other_team:  # No other opponent for team, break
                    break
                _teams_this_week.remove(other_team)
                _add_matchup(df, week_num, div_name, team, other_team)
    df["week"] = df["week"].astype(int)
    return df

def get_non_div_schedule():
    while 1:
        # there is a more optimized way of doing this but try this as many times as you get a middle schedule with no issues
        try:
            df = _get_non_div_schedule_wrapper()
            break
        except Exception as e:
            pass
    return df

def _get_non_div_schedule_wrapper():
    # initialize map and state
    team_to_div = _get_team_to_div()
    already_played = set()
    df = pd.DataFrame()
    
    for week_num in range(4, 12):    
        teams = list(itertools.chain.from_iterable([t for d, t in divisions.items()]))  # start with all teams
        played_this_week = []    
        while True:
            if len(teams) == 2:
                team, other_team = teams[0], teams[1]                        
                teams.remove(team)        
            else:    
                
                team = teams.pop()    
    
                # all matchups for team
                match_candidates = set((team, other) for other in _get_other_div(team))  # potential matchups for non division        
    
                # remove already played
                match_candidates -= already_played
                
                # at least one must have not been selected
                match_candidates = [t for t in match_candidates if t[0] in teams or t[1] in teams]

                # pick a random one
                matchup = list(match_candidates)[random.randint(0, len(match_candidates) - 1)]            
                other_team = matchup[0] if matchup[0] != team else matchup[1]
        
                # add to alread played
                played_this_week.append(team)
                played_this_week.append(other_team)
                        
            teams.remove(other_team)
            _add_matchup(df, week_num, "--", team, other_team)
            already_played.add((team, other_team))
            
            if not teams:
                break
    df["week"] = df["week"].astype(int)
    
    return df

def get_div_schedule(week_start=1):
    df = pd.DataFrame()
    for div_name, teams in divisions.items():
        df = pd.concat([df, _get_division_matches(teams, div_name, week_start)])
    return df.sort_values(["week", "div"]).reset_index(drop=True)

In [949]:
get_div_schedule()

,week,div,team_a,team_b
0,1,cam,tebro,broseph
1,1,cam,tebro,last
2,1,cam,tebro,aya
3,1,cin,plumpies,Teds
4,1,cin,plumpies,fubar
5,1,cin,plumpies,fearTheMex
6,1,cod,rum,kaptain
7,1,cod,rum,aguilar
8,1,cod,rum,pyong
9,2,cam,aya,last


In [951]:
get_non_div_schedule()

,week,div,team_a,team_b
0,4,--,broseph,plumpies
1,4,--,tebro,rum
2,4,--,aya,Teds
3,4,--,last,fearTheMex
4,4,--,kaptain,fubar
5,4,--,pyong,aguilar
6,5,--,broseph,fearTheMex
7,5,--,tebro,aguilar
8,5,--,aya,pyong
9,5,--,last,rum


In [953]:
get_div_schedule(week_start=12)

,week,div,team_a,team_b
0,12,cam,tebro,last
1,12,cam,tebro,aya
2,12,cam,tebro,broseph
3,12,cin,plumpies,fearTheMex
4,12,cin,plumpies,Teds
5,12,cin,plumpies,fubar
6,12,cod,rum,aguilar
7,12,cod,rum,kaptain
8,12,cod,rum,pyong
9,13,cam,aya,last
